# Relax Graphene with EMT

In [ ]:
import base64
from ase.io import write
from ase.build import make_supercell
from IPython.display import HTML
import io


def visualize_material_base64(material, title: str, rotation: str = '0x', number_of_repetitions: int = 1):
    """
    Returns an HTML string with a Base64-encoded image for visualization,
    including the name of the file, positioned horizontally.
    """
    # Set the number of unit cell repetition for the structure
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n, 0, 0], [0, n, 0], [0, 0, 1]])
    text = f"{material.symbols} - {title}"

    # Write image to a buffer to display in HTML
    buf = io.BytesIO()
    write(buf, material_repeat, format='png', rotation=rotation)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    html_str = f'''
    <div style="display: inline-block; margin: 10px; vertical-align: top;">
        <p>{text}</p>
        <img src="data:image/png;base64,{img_str}" alt="{title}" />
    </div>
    '''
    return html_str

## 1. Get original structure

In [ ]:

MATERIALS_FILES = [
    "C_Si_32_copy1.json"
]


In [ ]:
import json
from src.utils import to_pymatgen
from src.utils import ase_to_poscar, pymatgen_to_ase

materials_in = []
FOLDER = "uploads"
for file in MATERIALS_FILES:
    with open(f"{FOLDER}/{file}", "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]


input_material = pymatgen_materials[0]
ase_interface = pymatgen_to_ase(input_material)


html_input = visualize_material_base64(ase_interface, "input", "0x")
html_input_z = visualize_material_base64(ase_interface, "input_z", "-90x")
# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_input}{html_input_z}</div>'
display(HTML(html_content))


## 1. Add small random z displacements.
Add small random z displacements to allow for relaxation in z-direction, otherwise no force in z-direction can be calculated.

In [ ]:
import plotly.graph_objs as go
from IPython.display import display
from plotly.subplots import make_subplots


import numpy as np

# Set amplitude for z perturbation in Ångströms
amplitude = 0.1  # adjust as needed, e.g., 0.01 to 0.1 Å

positions = ase_interface.get_positions()
perturbed_positions = positions.copy()

# Apply random perturbation only to z
perturbed_positions[:, 2] += np.random.normal(-amplitude, amplitude, size=positions.shape[0]) + 5

ase_interface.set_positions(perturbed_positions)

ase_original_interface = ase_interface.copy()
html_original = visualize_material_base64(ase_original_interface, "Original material with z displacement", "-90x")
# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_original}</div>'
display(HTML(html_content))

In [ ]:
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.constraints import FixCartesian

RELAXATION_PARAMETERS = {
    "FMAX": 0.005,
}


# Apply constraint to all atoms to fix x and y
constraints = [FixCartesian(i, [True, True, False]) for i in range(len(ase_interface))]
ase_interface.set_constraint(constraints)


# Set up the calculator
calculator = EMT()
ase_interface.set_calculator(calculator)
dyn = FIRE(ase_interface)

# Initialize empty lists to store steps and energies
steps = []
energies = []

# Create a plotly figure widget
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
scatter = go.Scatter(x=[], y=[], mode='lines+markers', name='Energy')
fig.add_trace(scatter)
fig.update_layout(title_text='Real-time Optimization Progress', xaxis_title='Step', yaxis_title='Energy (eV)')

# Display figure widget
f = go.FigureWidget(fig)
display(f)


# Define a callback function to update the plot at each step
def plotly_callback():
    step = dyn.nsteps
    energy = ase_interface.get_total_energy()

    # Add the new step and energy to the lists
    steps.append(step)
    energies.append(energy)

    print(f"Step: {step}, Energy: {energy:.4f} eV")

    # Update the figure with the new data
    with f.batch_update():
        f.data[0].x = steps
        f.data[0].y = energies


# Run the relaxation
dyn.attach(plotly_callback, interval=1)
dyn.run(fmax=RELAXATION_PARAMETERS["FMAX"], steps =50)

# Extract results
# ase_original_interface = pymatgen_to_ase(interface)
# ase_original_interface.set_calculator(calculator)
ase_final_interface = ase_interface

# original_energy = ase_original_interface.get_total_energy()
relaxed_energy = ase_interface.get_total_energy()

# Print out the final relaxed structure and energy
# print('Original structure:\n', ase_to_poscar(ase_original_interface))
# print('\nRelaxed structure:\n', ase_to_poscar(ase_final_interface))
# print(f"The final energy is {float(relaxed_energy):.3f} eV.")

### 3. View structure before and after relaxation

In [ ]:

html_original = visualize_material_base64(ase_original_interface, "original", "0x")
html_original_z = visualize_material_base64(ase_original_interface, "original", "-90x")
html_relaxed = visualize_material_base64(ase_final_interface, "relaxed", "0x")
html_relaxed_z = visualize_material_base64(ase_final_interface, "relaxed", "-90x")

# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_original}{html_relaxed}</div>'
display(HTML(html_content))

In [ ]:
# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_original_z}{html_relaxed_z}</div>'
display(HTML(html_content))

In [ ]:
import numpy as np

# Get atomic positions from both interfaces
original_positions = ase_original_interface.get_positions()
final_positions = ase_final_interface.get_positions()

# Extract z-coordinates
original_z = original_positions[:, 2]
final_z = final_positions[:, 2]

# Compute delta_z for each atom
delta_z = final_z - original_z  # shape: (N,)
print(delta_z)

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.stats import binned_statistic_2d

# Step 1: Get original and final positions
original_pos = ase_original_interface.get_positions()
final_pos = ase_final_interface.get_positions()

# Step 2: Calculate Δz
delta_z = final_pos[:, 2] - original_pos[:, 2]
delta_z_abs = np.abs(delta_z)

# Step 3: Define 2D grid binning over x and y
x = original_pos[:, 0]
y = original_pos[:, 1]
z = delta_z_abs

# Adjust number of bins to suit resolution
n_bins_x = 150
n_bins_y = 150

# Step 4: Bin Δz over 2D x/y using mean (you can also use 'max', 'sum', etc.)
stat, x_edges, y_edges, _ = binned_statistic_2d(
    x, y, z,
    statistic='mean',
    bins=[n_bins_x, n_bins_y]
)

# Step 5: Create heatmap
heatmap = go.Heatmap(
    z=stat.T,  # Transpose to align x and y correctly
    x=(x_edges[:-1] + x_edges[1:]) / 2,
    y=(y_edges[:-1] + y_edges[1:]) / 2,
    colorscale='Thermal',
    colorbar=dict(title='Δz (Å)'),
)

layout = go.Layout(
    title='Δz Displacement Heatmap over x–y',
    xaxis=dict(title='X (Å)'),
    yaxis=dict(title='Y (Å)'),
    height=800,
    width=800,
)

fig = go.Figure(data=[heatmap], layout=layout)
fig.show()


In [ ]:
from mat3ra.made.material import Material
from mat3ra.made.tools.convert import from_ase
mat3ra_material = Material.create(from_ase(ase_final_interface))
# visualize_materials([mat3ra_material], viewer="wave")

from utils.jupyterlite import set_materials
n_atoms = len(mat3ra_material.basis.elements.values)
mat3ra_material.name = f"C{n_atoms}, Relaxed with EMT"
set_materials(mat3ra_material)